In [1]:
# Inherited: `data` 
#%run "C:\Users\Asus\Desktop\Coding\API_Flask\side_BenckMark\embedding_Workspace\bench\2_Preprocess.ipynb" 

import dill
# Load session 
dill.load_session('embedded_Data.pkl') 

In [2]:
data.head(2)

,Unnamed: 0,ProductId,UserId,Score,Summary,Text,combined,n_tokens,embedding,review_Embedding,semantic_Label,label_Embedding
0,0,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,Title: where does one start...and stop... wit...,52,"[0.007018072064965963, -0.02731654793024063, 0...","[0.007018072064965963, -0.02731654793024063, 0...",positive,"[-0.006508708465844393, -0.008743545040488243,..."
1,297,B003VXHGPK,A21VWSCGW7UUAR,4,"Good, but not Wolfgang Puck good","Honestly, I have to admit that I expected a li...","Title: Good, but not Wolfgang Puck good; Conte...",178,"[-0.003140551969408989, -0.009995664469897747,...","[-0.003140551969408989, -0.009995664469897747,...",positive,"[-0.006508708465844393, -0.008743545040488243,..."


### `Predictions`

---
### `Main Calculations` 

#### `Setup` 

##### `Classification Report` 
* methods: "`create_Report`", "`extract_summary_metrics`" 

In [3]:
from sklearn.metrics import classification_report 

In [4]:
def create_Report(current_Prediction):
    global cosine_Report 
    
    # Get the classification report with zero_division=1 to handle division by zero
    report = classification_report(data["semantic_Label"], data[current_Prediction], zero_division=1)
    
    title_List, precision_List, recall_List, f1_List, support_List = [], [], [], [], [] 

    for line in report.split("\n")[2:-5]:  # Skip the first two lines and the last five lines of the report
        try:
            # Split the line by spaces but exclude empty splits
            values = [value for value in line.split("  ") if value]
            
            title, precision, recall, f1, support = values
            title_List.append(title.strip())
            precision_List.append(float(precision))
            recall_List.append(float(recall))
            f1_List.append(float(f1))
            support_List.append(int(support))
        except:
            pass 

    # Create dataframe from the values 
    cosine_Report = pd.DataFrame({
        'title': title_List, 
        'precision': precision_List, 
        'recall': recall_List, 
        'f1': f1_List, 
        'support': support_List
    })    
    
    return cosine_Report


In [5]:
def extract_summary_metrics(current_Prediction):
    # Get the classification report with zero_division=1 to handle division by zero
    report = classification_report(data["semantic_Label"], data[current_Prediction], zero_division=1)
    
    # Split the report by lines
    lines = report.split("\n")
    
    # Extract accuracy
    try:
        accuracy = float(lines[-2].split(":")[1].strip())
    except:
        accuracy = float('NaN')
    
    # Extract macro avg metrics
    macro_avg = lines[-4].split()
    try:
        macro_precision = float(macro_avg[2])
    except:
        macro_precision = float('NaN')
    try:
        macro_recall = float(macro_avg[3])
    except:
        macro_recall = float('NaN')
    try:
        macro_f1 = float(macro_avg[4])
    except:
        macro_f1 = float('NaN')
    try:
        macro_support = int(macro_avg[5])
    except:
        macro_support = float('NaN')
    
    # Extract weighted avg metrics
    weighted_avg = lines[-3].split()
    try:
        weighted_precision = float(weighted_avg[2])
    except:
        weighted_precision = float('NaN')
    try:
        weighted_recall = float(weighted_avg[3])
    except:
        weighted_recall = float('NaN')
    try:
        weighted_f1 = float(weighted_avg[4])
    except:
        weighted_f1 = float('NaN')
    try:
        weighted_support = int(weighted_avg[5])
    except:
        weighted_support = float('NaN')
    
    # Create a DataFrame with the extracted metrics
    summary_df = pd.DataFrame({
        'title': ['accuracy', 'macro avg', 'weighted avg'],
        'precision': [float('NaN'), macro_precision, weighted_precision],
        'recall': [float('NaN'), macro_recall, weighted_recall],
        'f1': [float('NaN'), macro_f1, weighted_f1],
        'support': [accuracy, macro_support, weighted_support]
    })
    
    return summary_df


##### `Similarity Calculations`
* method: "`label_Similarity_Score`", "`predict_1`", "`predict_2`"

In [6]:
from scipy.spatial import distance

def label_Similarity_Score(review_embedding, label_embeddings, method='Cosine'):
    scores = {}
    
    for label, embedding in label_embeddings.items():
        if method == 'Cosine':
            scores[label] = 1 - distance.cosine(review_embedding, embedding)
        elif method == 'Euclidean':
            scores[label] = 1 / (1 + distance.euclidean(review_embedding, embedding))
        elif method == 'Manhattan':
            scores[label] = 1 / (1 + distance.cityblock(review_embedding, embedding))
        elif method == 'Minkowski':
            scores[label] = 1 / (1 + distance.minkowski(review_embedding, embedding, p=3))
        elif method == 'Pearson':
            scores[label] = np.corrcoef(review_embedding, embedding)[0, 1] # Pearson is not a distance metric, so we use correlation coefficient instead 
        elif method == 'Canberra':
            scores[label] = 1 / (1 + distance.canberra(review_embedding, embedding))
       
        else:
            raise ValueError(f"Unknown method: {method}")
    
    return scores

def predict_1(similarity_scores, n=2):
    # Sort labels based on scores and get the top n labels
    sorted_labels = sorted(similarity_scores, key=similarity_scores.get, reverse=True)
    return sorted_labels[:n][0]

def predict_2(scores):
    max_index = np.argmax(scores)
    if max_index < len(current_Labels):
        return current_Labels[max_index]
    else:
        return 'External'  # or any other placeholder 

#### `Process` 

##### `Similarities & Predictions` 
* Output: "`Probabilities`" & "`Labels`" 

In [7]:
# Get Similarity_Score for each review
data["Similarity_Score_Cosine"]    = data["review_Embedding"].apply(lambda x: label_Similarity_Score(x, label_Dict, method='Cosine')) 
data["Similarity_Score_Euclidean"] = data["review_Embedding"].apply(lambda x: label_Similarity_Score(x, label_Dict, method='Euclidean')) 
data["Similarity_Score_Manhattan"] = data["review_Embedding"].apply(lambda x: label_Similarity_Score(x, label_Dict, method='Manhattan')) 
data["Similarity_Score_Minkowski"] = data["review_Embedding"].apply(lambda x: label_Similarity_Score(x, label_Dict, method='Minkowski')) 
data["Similarity_Score_Pearson"]   = data["review_Embedding"].apply(lambda x: label_Similarity_Score(x, label_Dict, method='Pearson')) 
data["Similarity_Score_Canberra"]  = data["review_Embedding"].apply(lambda x: label_Similarity_Score(x, label_Dict, method='Canberra')) 

# Assign top 2 labels for each review 
data["predicted_labels_Cosine"]    = data["Similarity_Score_Cosine"].apply(lambda x: predict_1(x, 1))
data["predicted_labels_Euclidean"] = data["Similarity_Score_Euclidean"].apply(lambda x: predict_1(x, 1))
data["predicted_labels_Manhattan"] = data["Similarity_Score_Manhattan"].apply(lambda x: predict_1(x, 1))
data["predicted_labels_Minkowski"] = data["Similarity_Score_Minkowski"].apply(lambda x: predict_1(x, 1))
data["predicted_labels_Pearson"]   = data["Similarity_Score_Pearson"].apply(lambda x: predict_1(x, 1))
data["predicted_labels_Canberra"]  = data["Similarity_Score_Canberra"].apply(lambda x: predict_1(x, 1))

---
### `Report` 

#### `Process:` Counts

In [8]:
# Count values 
count_Cosine    = data["predicted_labels_Cosine"].value_counts() 
count_Euclidean = data["predicted_labels_Euclidean"].value_counts() 
count_Manhattan = data["predicted_labels_Manhattan"].value_counts() 
count_Minkowski = data["predicted_labels_Minkowski"].value_counts()  
count_Pearson = data["predicted_labels_Pearson"].value_counts()  
count_Canberra = data["predicted_labels_Canberra"].value_counts()  
# Total
count_Combined  = pd.DataFrame({"Cosine": count_Cosine.values,       "Euclidean": count_Euclidean.values, 
                                "Manhattan": count_Manhattan.values, "Minkowski": count_Minkowski.values, 
                                "Pearson": count_Pearson,            "Canberra": count_Canberra}, index = list(count_Cosine.index))
## Add sum row to the end of the dataframe
count_Combined.loc['Total',:]= count_Combined.sum(axis=0) 

#### `Process:` Means

In [9]:
# Mean Values
mean_Cosine, mean_Euclidean, mean_Manhattan, mean_Minkowski, mean_Pearson, mean_Canberra = [], [], [], [], [], [] 
for label in current_Labels: 
    mean_Cosine.append( data["Similarity_Score_Cosine"].apply(lambda x: x[label]).mean() ) 
    mean_Euclidean.append( data["Similarity_Score_Euclidean"].apply(lambda x: x[label]).mean() ) 
    mean_Manhattan.append( data["Similarity_Score_Manhattan"].apply(lambda x: x[label]).mean() ) 
    mean_Minkowski.append( data["Similarity_Score_Minkowski"].apply(lambda x: x[label]).mean() ) 
    mean_Pearson.append( data["Similarity_Score_Pearson"].apply(lambda x: x[label]).mean() )  
    mean_Canberra.append( data["Similarity_Score_Canberra"].apply(lambda x: x[label]).mean() )  
# Combined 
mean_Combined = pd.DataFrame({"Cosine": mean_Cosine,       "Euclidean": mean_Euclidean, 
                              "Manhattan": mean_Manhattan, "Minkowski": mean_Minkowski, 
                              "Pearson": mean_Pearson,     "Canberra": mean_Canberra, 
                              }, index = list(current_Labels))
## Add sum row to the end of the dataframe
mean_Combined.loc['Total',:]= mean_Combined.mean(axis=0)

#### `Process:` Precision, Recall, F1, Support  

In [15]:
titles = create_Report("predicted_labels_Cosine")['title'].tolist() 

In [38]:
report_Cosine = create_Report("predicted_labels_Cosine")   
report_Cosine['title'] = ['Cosines: ' + x for x in current_Labels]
# Add total row to the end of the dataframe
temp = report_Cosine.iloc[:,1:].mean(axis=0) 
temp = ['Total'] + list (temp)  
# Add temp to the end of the dataframe as a row
report_Cosine.loc[len(report_Cosine)] = temp  

In [39]:
report_Minkowski = create_Report("predicted_labels_Minkowski")   
report_Minkowski['title'] = ['Minkowski: ' + x for x in current_Labels]
# Add total row to the end of the dataframe
temp = report_Minkowski.iloc[:,1:].mean(axis=0) 
temp = ['Total'] + list (temp)  
# Add temp to the end of the dataframe as a row
report_Minkowski.loc[len(report_Minkowski)] = temp

In [40]:
report_Manhattan = create_Report("predicted_labels_Manhattan")   
report_Manhattan['title'] = ['Manhattan: ' + x for x in current_Labels]
# Add total row to the end of the dataframe
temp = report_Manhattan.iloc[:,1:].mean(axis=0) 
temp = ['Total'] + list (temp)  
# Add temp to the end of the dataframe as a row
report_Manhattan.loc[len(report_Manhattan)] = temp

In [41]:
report_Euclidean = create_Report("predicted_labels_Euclidean")   
report_Euclidean['title'] = ['Euclidean: ' + x for x in current_Labels]
# Add total row to the end of the dataframe
temp = report_Euclidean.iloc[:,1:].mean(axis=0) 
temp = ['Total'] + list (temp)  
# Add temp to the end of the dataframe as a row
report_Euclidean.loc[len(report_Euclidean)] = temp

In [42]:
report_Pearson = create_Report("predicted_labels_Pearson")   
report_Pearson['title'] = ['Pearson: ' + x for x in current_Labels]
# Add total row to the end of the dataframe
temp = report_Pearson.iloc[:,1:].mean(axis=0) 
temp = ['Total'] + list (temp)  
# Add temp to the end of the dataframe as a row
report_Pearson.loc[len(report_Pearson)] = temp

In [43]:
report_Canberra = create_Report("predicted_labels_Canberra")   
report_Canberra['title'] = ['Canberra: ' + x for x in current_Labels]
# Add total row to the end of the dataframe
temp = report_Canberra.iloc[:,1:].mean(axis=0) 
temp = ['Total'] + list (temp)  
# Add temp to the end of the dataframe as a row
report_Canberra.loc[len(report_Canberra)] = temp

In [44]:
top = pd.concat([report_Cosine, report_Minkowski], axis=1) 
bottom = pd.concat([report_Manhattan, report_Euclidean], axis=1) 
bottom2 = pd.concat([report_Pearson, report_Canberra], axis=1) 
combined = pd.concat([top, bottom], axis=0) 
combined = pd.concat([combined, bottom2], axis=0) 

#### `Process:` Accuracy, Macro Avg, Weighted Avg

In [45]:
titles_2 = extract_summary_metrics("predicted_labels_Cosine")['title'].tolist()  

In [53]:
extract_summary_metrics("predicted_labels_Cosine") 

,title,precision,recall,f1,support
0,accuracy,NaN,NaN,NaN,NaN
1,macro avg,925.0,NaN,NaN,NaN
2,weighted avg,0.8,0.89,0.83,925.0


In [54]:
report_2_Cosine             = extract_summary_metrics("predicted_labels_Cosine") 
report_2_Cosine['title']    = ['Cosine: ' + x for x in report_2_Cosine['title'] ] 
report_2_Minkowski          = extract_summary_metrics("predicted_labels_Minkowski") 
report_2_Minkowski['title'] = ['Minkowski: ' + x for x in report_2_Cosine['title'] ] 
report_2_Manhattan          = extract_summary_metrics("predicted_labels_Manhattan") 
report_2_Manhattan['title'] = ['Manhattan: ' + x for x in report_2_Cosine['title'] ] 
report_2_Euclidean          = extract_summary_metrics("predicted_labels_Euclidean") 
report_2_Euclidean['title'] = ['Euclidean: ' + x for x in report_2_Cosine['title'] ] 
report_2_Pearson            = extract_summary_metrics("predicted_labels_Pearson") 
report_2_Pearson['title']   = ['Pearson: ' + x for x in report_2_Cosine['title'] ] 
report_2_Canberra           = extract_summary_metrics("predicted_labels_Canberra") 
report_2_Canberra['title']  = ['Canberra: ' + x for x in report_2_Cosine['title'] ] 


In [55]:
top = pd.concat([report_2_Cosine, report_2_Minkowski], axis=1) 
bottom = pd.concat([report_2_Manhattan, report_2_Euclidean], axis=1) 
bottom2 = pd.concat([report_2_Pearson, report_2_Canberra], axis=1) 
combined_2 = pd.concat([top, bottom], axis=0) 
combined_2 = pd.concat([combined_2, bottom2], axis=0) 
combined_2.columns = (['title'] + titles_2 + ['X']) * 2 

---
### `Result`

In [56]:
mean_Combined

,Cosine,Euclidean,Manhattan,Minkowski,Pearson,Canberra
positive,0.737207,0.579858,0.042425,0.799796,0.736988,0.001088
negative,0.729144,0.576163,0.041937,0.797066,0.728925,0.001085
Total,0.733175,0.578010,0.042181,0.798431,0.732956,0.001086


In [57]:
count_Combined

,Cosine,Euclidean,Manhattan,Minkowski,Pearson,Canberra
positive,730.0,730.0,703.0,738.0,730.0,550.0
negative,195.0,195.0,222.0,187.0,195.0,375.0
Total,925.0,925.0,925.0,925.0,925.0,925.0


In [58]:
combined

,title,precision,recall,f1,support,title,precision,recall,f1,support
0,Cosines: positive,0.62,0.88,0.730,136.0,Minkowski: positive,0.650,0.890,0.750,136.0
1,Cosines: negative,0.98,0.90,0.940,789.0,Minkowski: negative,0.980,0.920,0.950,789.0
2,Total,0.80,0.89,0.835,462.5,Total,0.815,0.905,0.850,462.5
0,Manhattan: positive,0.51,0.84,0.640,136.0,Euclidean: positive,0.620,0.880,0.730,136.0
1,Manhattan: negative,0.97,0.86,0.910,789.0,Euclidean: negative,0.980,0.900,0.940,789.0
2,Total,0.74,0.85,0.775,462.5,Total,0.800,0.890,0.835,462.5
0,Pearson: positive,0.62,0.88,0.730,136.0,Canberra: positive,0.330,0.920,0.490,136.0
1,Pearson: negative,0.98,0.90,0.940,789.0,Canberra: negative,0.980,0.680,0.810,789.0
2,Total,0.80,0.89,0.835,462.5,Total,0.655,0.800,0.650,462.5


In [59]:
combined_2

,title,accuracy,macro avg,weighted avg,X,title,accuracy,macro avg,weighted avg,X
0,Cosine: accuracy,NaN,NaN,NaN,NaN,Minkowski: Cosine: accuracy,NaN,NaN,NaN,NaN
1,Cosine: macro avg,925.00,NaN,NaN,NaN,Minkowski: Cosine: macro avg,925.00,NaN,NaN,NaN
2,Cosine: weighted avg,0.80,0.89,0.83,925.0,Minkowski: Cosine: weighted avg,0.81,0.90,0.85,925.0
0,Manhattan: Cosine: accuracy,NaN,NaN,NaN,NaN,Euclidean: Cosine: accuracy,NaN,NaN,NaN,NaN
1,Manhattan: Cosine: macro avg,925.00,NaN,NaN,NaN,Euclidean: Cosine: macro avg,925.00,NaN,NaN,NaN
2,Manhattan: Cosine: weighted avg,0.74,0.85,0.77,925.0,Euclidean: Cosine: weighted avg,0.80,0.89,0.83,925.0
0,Pearson: Cosine: accuracy,NaN,NaN,NaN,NaN,Canberra: Cosine: accuracy,NaN,NaN,NaN,NaN
1,Pearson: Cosine: macro avg,925.00,NaN,NaN,NaN,Canberra: Cosine: macro avg,925.00,NaN,NaN,NaN
2,Pearson: Cosine: weighted avg,0.80,0.89,0.83,925.0,Canberra: Cosine: weighted avg,0.66,0.80,0.65,925.0


---
### `Checkpoint` 

In [60]:
import dill

# Save the entire session
dill.dump_session('session_A.pkl')  
# Actialize the session
#dill.load_session('session.pkl') 